In [1]:
#| default_exp templates

# PBS Job Templates
> Generate PBS submission scripts for VASP calculations with chasqui workflow integration

This module creates PBS scripts that:
- Execute VASP calculations with proper environment setup
- Handle job completion (success/failure)
- Write completion flags for status tracking
- Trigger the remote agent to submit waiting jobs
- Log execution details

## Template Structure

Each generated PBS script includes:

1. **PBS Directives** - Resource allocation (#PBS -l, -A, etc.)
2. **Environment Setup** - Module loads, ulimit, OMP settings
3. **VASP Execution** - MPI run with proper parameters
4. **Completion Handler** - Always runs, even on failure
5. **Agent Trigger** - Starts next job submission cycle

## Integration with Workflow
```
Job Script Template
       ↓
   PBS Queue
       ↓
  VASP Runs
       ↓
Completion Handler → Writes flag → Triggers agent.sh
                                          ↓
                                   Submits next jobs
```

In [2]:
#| export
from string import Template
from pathlib import Path
from typing import Optional, Dict, Any
from datetime import datetime

## PBS Script Template

Based on your existing script with enhancements for workflow automation.

In [4]:
#| export

PBS_TEMPLATE = """#!/bin/bash

#PBS -N $JOB_NAME
#PBS -l select=$CORES:ncpus=36:mpiprocs=36
#PBS -A $PROJECT
#PBS -l walltime=$TIME
#PBS -j oe
#PBS -o $JOB_NAME.out

# Job metadata for chasqui
JOB_ID="$JOB_ID"
CHASQUI_DIR="$CHASQUI_DIR"

cd $$PBS_O_WORKDIR
NNODES=`wc -l < $$PBS_NODEFILE`
echo "======================================"
echo "Job: $JOB_NAME"
echo "Job ID: $$PBS_JOBID"
echo "Chasqui ID: $$JOB_ID"
echo "Nodes: $$NNODES"
echo "Started: $$(date)"
echo "======================================"

# Environment setup
ulimit -s unlimited
export OMP_NUM_THREADS=1

# Load modules
module purge
module load vasp/6.4.3

# Run VASP
echo "Running VASP..."
mpirun -np $$NNODES $VASP

# Capture exit code
EXIT_CODE=$$?

echo "======================================"
echo "VASP finished with exit code: $$EXIT_CODE"
echo "Completed: $$(date)"
echo "======================================"

# ============================================
# CHASQUI COMPLETION HANDLER (always runs)
# ============================================

if [ $$EXIT_CODE -eq 0 ]; then
    STATUS="DONE"
    echo "✓ Job completed successfully"
else
    STATUS="FAIL"
    echo "✗ Job failed with exit code $$EXIT_CODE"
fi

# Write completion flag
COMPLETED_DIR="$$CHASQUI_DIR/completed"
mkdir -p $$COMPLETED_DIR
echo "$$STATUS" > $$COMPLETED_DIR/$${JOB_ID}.flag
echo "$$PBS_JOBID" >> $$COMPLETED_DIR/$${JOB_ID}.flag
echo "$$(date)" >> $$COMPLETED_DIR/$${JOB_ID}.flag

# Move job script to completed
SUBMITTED_DIR="$$CHASQUI_DIR/submitted"
mv $$SUBMITTED_DIR/$${JOB_ID}.sh $$COMPLETED_DIR/ 2>/dev/null

# Log completion
AGENT_LOG="$$CHASQUI_DIR/logs/agent.log"
echo "$$(date -Iseconds) JOB_COMPLETE job=$$JOB_ID pbs=$$PBS_JOBID status=$$STATUS exit_code=$$EXIT_CODE" >> $$AGENT_LOG

# Trigger agent to submit waiting jobs (with file lock)
echo "Triggering agent to submit next jobs..."
flock -n $$CHASQUI_DIR/agent.lock -c "bash $$CHASQUI_DIR/agent.sh" &

# Exit with original VASP exit code
exit $$EXIT_CODE
"""

In [5]:
#| export

def generate_pbs_script(
    job_id: str,
    job_name: Optional[str] = None,
    cores: int = 1,
    walltime: str = "48:00:00",
    project: str = "AARC1",
    vasp_version: str = "vasp_gam",
    chasqui_remote_dir: str = "~/chasqui_remote",
    output_path: Optional[str] = None
) -> str:
    """
    Generate PBS submission script for VASP job.
    
    Args:
        job_id: Unique job identifier (UUID from database)
        job_name: Human-readable job name (default: job_id)
        cores: Number of compute nodes to request (default: 1)
        walltime: Maximum runtime in HH:MM:SS format (default: "48:00:00")
        project: PBS account/project code (default: "AARC1")
        vasp_version: VASP executable name (default: "vasp_gam")
        chasqui_remote_dir: Remote chasqui directory (default: "~/chasqui_remote")
        output_path: If provided, write script to this file
        
    Returns:
        PBS script content as string
        
    Example:
        >>> script = generate_pbs_script(
        ...     job_id="abc-123-def",
        ...     job_name="Au_bulk_relax",
        ...     cores=2,
        ...     walltime="24:00:00",
        ...     project="MyProject",
        ...     vasp_version="vasp_std"
        ... )
        >>> print(script[:50])
        #!/bin/bash
        
        #PBS -N Au_bulk_relax
    """
    # Use job_id as name if not provided
    if job_name is None:
        job_name = f"chasqui_{job_id[:8]}"
    
    # Substitute template variables
    template = Template(PBS_TEMPLATE)
    script = template.safe_substitute(
        JOB_ID=job_id,
        JOB_NAME=job_name,
        CORES=cores,
        TIME=walltime,
        PROJECT=project,
        VASP=vasp_version,
        CHASQUI_DIR=chasqui_remote_dir
    )
    
    # Write to file if requested
    if output_path:
        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        output_file.write_text(script)
    
    return script

## Job-Specific Script Generation

Generate script directly from database job entry.

In [6]:
#| export

def generate_pbs_script_from_job(
    job: Dict[str, Any],
    output_path: Optional[str] = None
) -> str:
    """
    Generate PBS script from database job entry.
    
    Args:
        job: Job dictionary from ChasquiDB.get_job()
        output_path: Optional path to write script
        
    Returns:
        PBS script content as string
        
    Example:
        >>> from chasqui.database import ChasquiDB
        >>> db = ChasquiDB()
        >>> job = db.get_job("abc-123")
        >>> script = generate_pbs_script_from_job(job)
    """
    import json
    
    # Extract VASP config if present
    vasp_config = {}
    if job.get('vasp_config'):
        vasp_config = json.loads(job['vasp_config'])
    
    # Generate script with config parameters
    return generate_pbs_script(
        job_id=job['job_id'],
        job_name=vasp_config.get('job_name'),
        cores=vasp_config.get('cores', 1),
        walltime=vasp_config.get('walltime', '48:00:00'),
        project=vasp_config.get('project', 'AARC1'),
        vasp_version=vasp_config.get('vasp_version', 'vasp_gam'),
        chasqui_remote_dir=vasp_config.get('chasqui_remote_dir', '~/chasqui_remote'),
        output_path=output_path
    )

## Script Validation

Helper to validate generated scripts.

In [8]:
#| export

def validate_pbs_script(script: str) -> bool:
    """
    Validate that PBS script has required elements.
    
    Args:
        script: PBS script content
        
    Returns:
        True if script looks valid
        
    Raises:
        ValueError: If script is missing required elements
    """
    required_elements = [
        '#!/bin/bash',
        '#PBS -N',
        '#PBS -l select=',
        '#PBS -l walltime=',
        'module load vasp',
        'mpirun',
        'EXIT_CODE',
        'DONE',
        'FAIL',
        'agent.sh'
    ]
    
    missing = []
    for element in required_elements:
        if element not in script:
            missing.append(element)
    
    if missing:
        raise ValueError(f"PBS script missing required elements: {missing}")
    
    return True

## Tests

Verify script generation works correctly.

In [9]:
#| hide
import tempfile
import os

# Test basic script generation
script = generate_pbs_script(
    job_id="test-job-123",
    job_name="test_vasp",
    cores=2,
    walltime="12:00:00",
    project="TestProject",
    vasp_version="vasp_std"
)

print("Generated script length:", len(script))
assert len(script) > 500, "Script too short"
assert "test_vasp" in script, "Job name not in script"
assert "test-job-123" in script, "Job ID not in script"
assert "select=2:" in script, "Cores not set correctly"
assert "12:00:00" in script, "Walltime not set correctly"
assert "#PBS -A TestProject" in script, "Project not set correctly"
assert "vasp_std" in script, "VASP version not set correctly"
print("✓ Basic generation works")

# Test validation
try:
    validate_pbs_script(script)
    print("✓ Script validation passes")
except ValueError as e:
    print(f"✗ Validation failed: {e}")

# Test file writing
with tempfile.NamedTemporaryFile(mode='w', suffix='.sh', delete=False) as f:
    temp_path = f.name

try:
    script = generate_pbs_script(
        job_id="file-test-456",
        output_path=temp_path
    )
    
    assert os.path.exists(temp_path), "File not created"
    with open(temp_path) as f:
        content = f.read()
    assert "file-test-456" in content, "Job ID not in file"
    assert "#PBS -A AARC1" in content, "Default project not in file"
    print("✓ File writing works")
finally:
    os.unlink(temp_path)

# Test with job dictionary (simulated)
test_job = {
    'job_id': 'dict-job-789',
    'vasp_config': '{"job_name": "Au_relax", "cores": 4, "walltime": "06:00:00", "project": "CustomProj"}'
}

script = generate_pbs_script_from_job(test_job)
assert "Au_relax" in script, "Config job name not used"
assert "select=4:" in script, "Config cores not used"
assert "06:00:00" in script, "Config walltime not used"
assert "#PBS -A CustomProj" in script, "Config project not used"
print("✓ Generation from job dict works")

print("\n✅ All tests passed!")

Generated script length: 1938
✓ Basic generation works
✓ Script validation passes
✓ File writing works
✓ Generation from job dict works

✅ All tests passed!


## Usage Examples

### Basic Usage
```python
from chasqui.templates import generate_pbs_script

# Generate script with custom project
script = generate_pbs_script(
    job_id="abc-123-def-456",
    job_name="Au_bulk_optimization",
    cores=2,
    walltime="24:00:00",
    project="MyResearchProject",
    vasp_version="vasp_std"
)

# Write to file
with open("job.sh", "w") as f:
    f.write(script)
```

### With Database Integration
```python
from chasqui.database import ChasquiDB
from chasqui.templates import generate_pbs_script_from_job

# Create job in database
db = ChasquiDB()
job_id = db.create_job(
    local_path="/path/to/vasp",
    vasp_config={
        "job_name": "Si_bandstructure",
        "cores": 4,
        "walltime": "12:00:00",
        "project": "MATERIALS2024",
        "vasp_version": "vasp_std"
    }
)

# Generate script from job
job = db.get_job(job_id)
script = generate_pbs_script_from_job(
    job,
    output_path=f"jobs/{job_id}.sh"
)
```

### Using Default Project
```python
# If you omit project, it defaults to "AARC1"
script = generate_pbs_script(
    job_id="xyz-789",
    cores=8,
    walltime="72:00:00"
)
# Will use: #PBS -A AARC1
```

### Multiple Projects
```python
# Different jobs can use different projects
jobs = [
    {"project": "AARC1", "job_name": "job1"},
    {"project": "BioPhys", "job_name": "job2"},
    {"project": "MatSci", "job_name": "job3"},
]

for job_config in jobs:
    script = generate_pbs_script(
        job_id=f"job_{job_config['job_name']}",
        project=job_config['project'],
        **job_config
    )
```

In [11]:
from chasqui.templates import generate_pbs_script

# Generate a test script
script = generate_pbs_script(
    job_id="test-001",
    job_name="test_job",
    cores=1,
    walltime="01:00:00",
    vasp_version="vasp_gam"
)

# Print first 50 lines
print('\n'.join(script.split('\n')[:50]))

#!/bin/bash

#PBS -N test_job
#PBS -l select=1:ncpus=36:mpiprocs=36
#PBS -A AARC1
#PBS -l walltime=01:00:00
#PBS -j oe
#PBS -o test_job.out

# Job metadata for chasqui
JOB_ID="test-001"
CHASQUI_DIR="~/chasqui_remote"

cd $PBS_O_WORKDIR
NNODES=`wc -l < $PBS_NODEFILE`
echo "======================================"
echo "Job: test_job"
echo "Job ID: $PBS_JOBID"
echo "Chasqui ID: $JOB_ID"
echo "Nodes: $NNODES"
echo "Started: $(date)"
echo "======================================"

# Environment setup
ulimit -s unlimited
export OMP_NUM_THREADS=1

# Load modules
module purge
module load vasp/6.4.3

# Run VASP
echo "Running VASP..."
mpirun -np $NNODES vasp_gam

# Capture exit code
EXIT_CODE=$?

echo "======================================"
echo "VASP finished with exit code: $EXIT_CODE"
echo "Completed: $(date)"
echo "======================================"

# ============================================
# CHASQUI COMPLETION HANDLER (always runs)
# ============================================

